In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
from dateutil import parser
import sqlalchemy
from datetime import datetime

In [3]:
driver = webdriver.Chrome("chromedriver.exe")
broker=[] #List to store name of the broker
company_name=[] #List to store company names 
current_price=[] #List to store rating of the product
predict_date=[]
recomended_buying=[]
target_price=[]
driver.get("https://www.moneycontrol.com/news/tags/recommendations.html/page-934/")


In [5]:
content = driver.page_source
soup = BeautifulSoup(content,'html.parser')
Alldata=soup.select_one("div.righttexcontent2 div.right-table01 div.scr-result2")
# for row in Alldata.findAll('div',class_='middle-colum02'):
for row in Alldata.findAll('div',attrs={'class':'middle-colum02'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    

Test Cell

In [6]:
temp_data      #test

'31 Oct 19'

Classifying into different dataframes

In [7]:
df1 = pd.DataFrame({'broker':broker})  
df2 = pd.DataFrame({'company_name':company_name})
df3 = pd.DataFrame({'current_price':current_price})
df4 = pd.DataFrame({'recomended_buing':recomended_buying})
df5=pd.DataFrame({'target_price':target_price})
df6=pd.DataFrame({'predict_date':predict_date})
#dfALL = pd.DataFrame(broker,company_name,current_price,recomended_buying,target_price)


Cleaning Data

In [8]:
#splitting operation
rb=df4['recomended_buing'].str.split('Rs.').str[1] 
cp=df3['current_price'].str.split('Rs.').str[1]
tp=df5['target_price'].str.split('Rs.').str[1]
predict_date=df6['predict_date'].str.split('10').str[0]



In [9]:
dfall=pd.DataFrame({'broker':broker, 'company_name':company_name, 'current_price':cp, 'recomended_buying':rb, 'predict_date':predict_date, 'target_price':tp})
dfallclean = dfall[dfall['predict_date']!='']
dfallclean['predict_date'] =  pd.to_datetime(dfallclean['predict_date'], format='%d %b %y')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
dfallclean

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Axis Direct,Hindustan Unilever,1917.9,1928.75,2020-01-06,2175.00
1,HDFC Securities,Alkem Laboratories,2122.05,2089.44,2020-01-06,2650.00
2,Ashika Stock Broking,Alkem Laboratories,2122.05,2100.00,2020-01-06,2423.00
3,HDFC Securities,Karur Vysya Bank,55.9,56.85,2020-01-06,64.00
4,Nirmal Bang,UPL,590,595.00,2020-01-03,686.00
...,...,...,...,...,...,...
17426,ICICIdirect.com,Graphite India,290.55,285.47,2019-10-31,330.00
17427,Karvy Stock Broking,Cummins India,571.8,549.31,2019-10-31,545.00
17428,Karvy Stock Broking,Indoco Remedies,180,161.95,2019-10-31,189.00
17429,Geojit Financial Services,Kotak Mahindra Bank,1644,1587.58,2019-10-31,1682.00


In [11]:
dfallclean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16770 entries, 0 to 17430
Data columns (total 6 columns):
broker               16770 non-null object
company_name         16770 non-null object
current_price        16770 non-null object
recomended_buying    16770 non-null object
predict_date         16770 non-null datetime64[ns]
target_price         16770 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 917.1+ KB


In [12]:
database_username = 'smarsproddbuser@mbt-smars-mysql'
database_password = 'modern@1234'
database_ip       = 'mbt-smars-mysql.mysql.database.azure.com'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))
dfallclean.to_sql(con=database_connection, name='marketsmojorecos1', if_exists='append')

In [15]:
dfallclean1 = dfallclean.copy()

In [16]:
dfallclean1.drop_duplicates(inplace = True)
dfallclean1

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Axis Direct,Hindustan Unilever,1917.9,1928.75,2020-01-06,2175.00
1,HDFC Securities,Alkem Laboratories,2122.05,2089.44,2020-01-06,2650.00
2,Ashika Stock Broking,Alkem Laboratories,2122.05,2100.00,2020-01-06,2423.00
3,HDFC Securities,Karur Vysya Bank,55.9,56.85,2020-01-06,64.00
4,Nirmal Bang,UPL,590,595.00,2020-01-03,686.00
...,...,...,...,...,...,...
17426,ICICIdirect.com,Graphite India,290.55,285.47,2019-10-31,330.00
17427,Karvy Stock Broking,Cummins India,571.8,549.31,2019-10-31,545.00
17428,Karvy Stock Broking,Indoco Remedies,180,161.95,2019-10-31,189.00
17429,Geojit Financial Services,Kotak Mahindra Bank,1644,1587.58,2019-10-31,1682.00
